In [5]:
import nltk 
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd


#load dataset

data=gutenberg.raw('shakespeare-hamlet.txt')


with open('hamlet.txt','w') as file:
    file.write(data)


[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/hammadkhan/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [6]:
import numpy as np 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

with open('hamlet.txt','r') as file:
    text = file.read().lower()
    
    
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1



In [7]:
total_words

4818

In [8]:
# tokenizer.word_index

In [9]:
## input seq


inputsequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_seq=token_list[:i+1]
        inputsequences.append(n_gram_seq)

In [10]:
# inputsequences

In [11]:
## pad seq

max_seq_len=max([len(x) for x in inputsequences])
max_seq_len

14

In [12]:
inputsequences=np.array(pad_sequences(inputsequences,maxlen=max_seq_len,padding='pre'))

In [13]:
inputsequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [14]:
#predictors and labels 
import tensorflow as tf
x,y=inputsequences[:,:-1],inputsequences[:,-1]
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [15]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
x_train, x_test,y_train,y_test= train_test_split(x,y,test_size=0.2)

In [17]:
##training the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,Dense,Embedding

In [18]:
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_seq_len))  # Embedding layer
model.add(LSTM(150, return_sequences=True))  # First LSTM layer
model.add(Dropout(0.2))  # Dropout for regularization
model.add(LSTM(100))  # Second LSTM layer
model.add(Dense(total_words, activation="softmax"))  # Output layer

# Build the model with a sample input shape (batch_size, max_seq_len)
model.build(input_shape=(None, max_seq_len))  # Ensures model weights are created

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

# Print model summary
model.summary()


/Users/hammadkhan/Documents/GitHub/Simple-RNN-implementation/venv/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 14, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 14, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# history=model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1)


In [20]:
model.save('50_epochs_model.h5')

In [21]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('50_epochs_model.h5')
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
# Continue training for 10 more epochs
# history=model.fit(x_train,y_train,epochs=20,validation_data=(x_test,y_test),verbose=1)


In [22]:
def predict_next_word(model,tokenizer,text,max_seq_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_seq_len:
        token_list = token_list[-(max_seq_len-1):]
    token_list= pad_sequences([token_list],maxlen=max_seq_len-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted_word_ind= np.argmax(predicted,axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_ind:
            return word
    return None

In [34]:
input_text="Almost to Ielly"

print(f"input text: {input_text}")
max_seq_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_seq_len)
print(f"Next word predict : {next_word}")

input text: Almost to Ielly
Next word predict : knotty


In [24]:
from tensorflow.keras.models import load_model

# Load the saved model
model1 = load_model('100_epochs_model.h5')

In [32]:
input_text="Almost to Ielly with the Act of"
print(f"input text: {input_text}")
max_seq_len=model1.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_seq_len)
print(f"Next word predict : {next_word}")

input text: Almost to Ielly with the Act of
Next word predict : satyre


In [26]:
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)